<h1 style = "font-size: 35px; text-align: center;">AI Genetic Hands On</h1>
<h1 style = "font-size: 25px; text-align: center;">Hospital Job Scheduling for Doctors</h1>
<h1 style = "font-size: 25px; text-align: center; color: #666">Taha Shabani - Soheil Shirvani</h1>
<h4 style="text-align: center">Spring 1400</h4>

In [1]:
testFile = "test1.txt"

In [3]:
def readInput() :
    file = open(testFile, 'r+')
    fileList = file.readlines()
    fileList = [s.replace('\n', '') for s in fileList]
    
    [days, doctors] = [int(i) for i in fileList[0].split()]
    maxCapacity = int(fileList[1])
    offDays = int(fileList[2])
    
    allShifts = []
    for i in range(3, days + 3):
        dayRequirements = fileList[i].split()
        morningReqs = [int(i) for i in dayRequirements[0].split(",")]
        eveningReqs = [int(i) for i in dayRequirements[1].split(",")]
        nightReqs = [int(i) for i in dayRequirements[2].split(",")]
        
        allShifts.append((morningReqs, eveningReqs, nightReqs))
        
#     print(days, doctors, maxCapacity, offDays)
#     for i in allShifts:
#         print(i.printInfo())

    file.close()
    return [days, list(range(doctors)), maxCapacity, offDays, allShifts];

days, doctorsIds, maxCapacity, offDays, allShifts = readInput()

In [102]:
import numpy as np
from random import sample, randrange, shuffle, uniform

class JobScheduler:
    def __init__(self, fileInfo):
        self.days = fileInfo[0]
        self.doctors = len(fileInfo[1])
        self.doctorsIds = fileInfo[1]
        self.maxCapacity = fileInfo[2]
        self.offDays = fileInfo[3]
        self.allShifts = fileInfo[4]
        
        self.restartLimitation = 120 #consideration: if we stuck in local maximums this would be needed
        self.popSize = 500
        self.crossOverPoints = 2
        self.elitismPercentage = 10 #move x% best of parents directly to the new population
        self.pc = 0.6 #crossover probability
        self.pm = 0.2  #mutation probability
        
        self.chromosomeSet = {}
        self.chromosomes = self.getInitialChromosomes() #[0]: chromosome value, [1]: fitness value
#         print(self.chromosomes[0])
#         print(self.chromosomes[1])
#         self.crossOver(self.chromosomes[0], self.chromosomes[1])
#         print(self.chromosomes[0])
#         print(self.chromosomes[1])

#         display(self.chromosomes[0])
#         self.mutate(self.chromosomes[0])
#         display(self.chromosomes[0])
        
        
    
    
    def getInitialChromosomes(self):
        population = []
        for i in range(self.popSize):
            population.append(self.createChromosome())
        return population
    
    def createChromosome(self):
        allShiftIds = [[0 for i in range(3)] for j in range(self.days)]

        for day in range(self.days):
            [morningDoctors, eveningDoctors, nightDoctors] = sample(list(range(self.doctors + 1)), 3)
            
            allShiftIds[day][0] = sample(list(range(self.doctors)), morningDoctors)
            allShiftIds[day][1] = sample(list(range(self.doctors)), eveningDoctors)
            allShiftIds[day][2] = sample(list(range(self.doctors)), nightDoctors)

        return allShiftIds
    
    def crossOver(self, chrome1, chrome2):  
        points = sorted(sample(list(range(self.days)), self.crossOverPoints))
        for i in points:
            if uniform(0, 1) <= self.pc:
                temp = chrome1[i]
                chrome1[i] = chrome2[i]
                chrome2[i] = temp
                
    def mutate(self, chrome):
        if uniform(0, 1) <= self.pm:
            randomDay = randrange(self.days)        
            [morningDoctors, eveningDoctors, nightDoctors] = sample(list(range(self.doctors + 1)), 3)

            chrome[randomDay][0] = sample(list(range(self.doctors)), morningDoctors)
            chrome[randomDay][1] = sample(list(range(self.doctors)), eveningDoctors)
            chrome[randomDay][2] = sample(list(range(self.doctors)), nightDoctors)
            
    def calculateCost(self, chrome):
        nightShiftViolations = self.calculateNightShiftViolations(chrome)
        minMaxReqsViolations = self.calculateMinMaxReqsViolations(chrome)
        maxShiftViolations = self.calculateMaxShiftViolations()
        return nightShiftViolations + minMaxReqsViolations + maxShiftViolations
    
    def calculateNightShiftViolations(self, chrome):
        violations = 0
        
        #morning or evening shift after a night shift
        for day in range(len(chrome)-1):
            for nightDoctor in chrome[day][2]:
                if (nightDoctor in chrome[day+1][0]):
                    violations += 1
                if (nightDoctor in chrome[day+1][1]):
                    violations += 1
        
        #three night shifts in a row
        for day in range(len(chrome)-2):
            for nightDoctor in chrome[i][2]:
                if (nightDoctor in chrome[day+1][2] and nightDoctor in chrome[day+2][2]):
                    violations += 1
        
        return violations
    
    def calculateMinMaxReqsViolations(self, chrome):
        violations = 0
        
        for day in range(len(chrome)):
            for shift in range(3):
                shiftDoctors = len(chrome[day][shift])
                if (shift < allShifts[day][shift][0] or shift > allShifts[day][shift][0]):
                    violation += 1
        
        return violations
    
    def calculateMaxShiftViolations(self, chrome):
        violations = 0
        doctorShifts = [0 for i in doctorIds]
        
        for day in range(len(chrome)):
            for shift in range(3):
                for did in chrome[day][shift]:

        
        
        
        return violations
        
    
    def schedule(self):
        print()
        # todo
        

IndentationError: expected an indented block (<ipython-input-102-cb153079ae2b>, line 116)

In [98]:
import time

fileInfo = readInput()

start = time.time()

scheduler = JobScheduler(fileInfo)
scheduler.schedule()

end = time.time()

print("time: ", '%.2f'%(end - start), 'sec')

[[[], [0, 1, 3, 2], [2]],
 [[0, 1, 2, 3, 4], [], [0]],
 [[], [1, 0, 4], [0, 1]],
 [[2, 4, 1], [1], [2, 4]],
 [[2], [0, 4, 1, 3], [2, 3, 4, 0, 1]],
 [[0], [4, 3, 0, 2, 1], []],
 [[2, 1, 0, 4, 3], [0, 3, 1], [2]]]

0.02833134632016865


[[[], [0, 1, 3, 2], [2]],
 [[0, 1, 2, 3, 4], [], [0]],
 [[], [1, 0, 4], [0, 1]],
 [[2, 4, 1], [1], [2, 4]],
 [[2], [0, 4, 1, 3], [2, 3, 4, 0, 1]],
 [[0], [4, 3, 0, 2, 1], []],
 [[2, 1, 0, 4, 3], [0, 3, 1], [2]]]


time:  0.09 sec


# Read Input File

In [11]:
def readInput() :
    file = open(testFile, 'r+')
    fileList = file.readlines()
    fileList = [s.replace('\n', '') for s in fileList]
    
    [days, doctors] = [int(i) for i in fileList[0].split()]
    maxCapacity = int(fileList[1])
    offDays = int(fileList[2])
    
    allShifts = []
    for i in range(3, days + 3):
        dayRequirements = fileList[i].split()
        morningReqs = [int(i) for i in dayRequirements[0].split(",")]
        eveningReqs = [int(i) for i in dayRequirements[1].split(",")]
        nightReqs = [int(i) for i in dayRequirements[2].split(",")]
        
        allShifts.append((morningReqs, eveningReqs, nightReqs))
        
#     print(days, doctors, maxCapacity, offDays)
#     for i in allShifts:
#         print(i.printInfo())

    file.close()
    return [days, list(range(doctors)), maxCapacity, offDays, allShifts];

days, doctors_id, maxCapacity, offDays, allShifts = readInput()

days, doctors_id, maxCapacity, offDays, allShifts

(7,
 [0, 1, 2, 3, 4],
 8,
 2,
 [([2, 3], [1, 4], [2, 2]),
  ([1, 3], [1, 2], [3, 5]),
  ([3, 3], [2, 3], [1, 4]),
  ([1, 4], [1, 2], [1, 3]),
  ([1, 1], [0, 2], [2, 4]),
  ([4, 5], [1, 2], [1, 2]),
  ([1, 3], [1, 3], [2, 2])])

# Initial Choromosomes

In [22]:


def create_chromosome():
    choromosome = np.zeros(shape=(len(allShifts),3)).astype(int)
    all_shift_ids = [[0 for i in range(3)] for j in range(len(allShifts))]
    
    for day in range(len(allShifts)):
        [morningDoctors, eveningDoctors, nightDoctors] = sample(list(range(len(doctors_id)+1)), 3)
        
        all_shift_ids[day][0] = sample(list(range(len(doctors_id))), morningDoctors)
        all_shift_ids[day][1] = sample(list(range(len(doctors_id))), eveningDoctors)
        all_shift_ids[day][2] = sample(list(range(len(doctors_id))), nightDoctors)

    return all_shift_ids
create_chromosome()

[[[2, 3, 4], [], [2, 4, 0, 1, 3]],
 [[], [0, 1], [0, 2, 4]],
 [[1], [], [3, 2, 0, 4]],
 [[2, 1, 0, 3, 4], [], [2, 0, 4]],
 [[0, 4, 2], [], [0]],
 [[], [2, 3, 4], [1, 3]],
 [[], [0, 3, 1, 2, 4], [4]]]

# Initial Population

In [23]:
population_number = 50
population = []
for i in range(population_number):
    population.append(create_chromosome())
    
population

[[[[1], [4, 3, 2, 1, 0], [3, 4]],
  [[], [1], [3, 2, 0, 4]],
  [[3], [0, 2, 3, 1, 4], []],
  [[1], [4, 0, 1], []],
  [[0], [4, 2, 1, 0], [3, 0, 4, 1, 2]],
  [[3, 1, 0, 2], [1, 3, 2], [0, 2]],
  [[1], [3, 4, 2, 1, 0], []]],
 [[[3], [], [4, 0, 2, 1]],
  [[1, 4, 2], [0], [0, 3]],
  [[3, 1], [2, 3, 1], [4]],
  [[4, 1, 2, 3, 0], [1, 2, 0], [1]],
  [[2, 4, 0, 1, 3], [1, 4, 2], []],
  [[1, 0, 2], [0], [2, 0, 3, 1, 4]],
  [[0], [1, 0, 4, 2, 3], [2, 1, 0, 3]]],
 [[[], [4, 3, 2, 0], [4, 0, 3]],
  [[], [0, 3, 4], [0, 4]],
  [[0, 1, 4], [4], [1, 2, 4, 0, 3]],
  [[0, 2], [1, 2, 4], [4, 3, 1, 0, 2]],
  [[], [0, 2, 1, 3], [2, 3, 0]],
  [[], [1, 3, 4], [3, 4, 2, 0, 1]],
  [[2], [4, 2], [4, 2, 3, 0, 1]]],
 [[[0, 1], [0, 1, 3], [0]],
  [[4, 2, 3, 0], [1, 4, 3], [2, 0]],
  [[1, 4, 3, 2, 0], [0], [1, 2]],
  [[0, 2, 4, 3], [1, 0], []],
  [[0, 4, 2], [3], [4, 3, 1, 2]],
  [[0], [3, 0, 2], [2, 0, 1, 4]],
  [[0], [4, 3, 2, 1], []]],
 [[[1, 2, 0, 4], [], [2]],
  [[3], [3, 4, 0, 2, 1], [1, 4, 2]],
  [[1, 0], [2

# Cross Over

In [25]:
cross_over_chance = 0.6
crossOverPoints = 2
import random

def cross_over(chrome1, chrome2):
    # days_to_change = sorted(random.sample(range(len(allShifts)), days_to_cross_over))

#     chrome1 = population[0]
#     chrome2 = population[1]

    display(chrome1)
    points = sorted(sample(list(range(days)), crossOverPoints))
    print(points)
     
    for i in points:
        print(i)
        if random.uniform(0, 1) <= cross_over_chance:
            print("hey: ", i)
            temp = chrome1[i]
            chrome1[i] = chrome2[i]
            chrome2[i] = temp
        
#     display(chrome1)
    return chrome1, chrome2

cross_over(population[0], population[1])

[[[1], [4, 3, 2, 1, 0], [3, 4]],
 [[], [1], [3, 2, 0, 4]],
 [[3], [0, 2, 3, 1, 4], []],
 [[1], [4, 0, 1], []],
 [[0], [4, 2, 1, 0], [3, 0, 4, 1, 2]],
 [[3, 1, 0, 2], [1, 3, 2], [0, 2]],
 [[1], [3, 4, 2, 1, 0], []]]

[3, 5]
3
hey:  3
5


([[[1], [4, 3, 2, 1, 0], [3, 4]],
  [[], [1], [3, 2, 0, 4]],
  [[3], [0, 2, 3, 1, 4], []],
  [[4, 1, 2, 3, 0], [1, 2, 0], [1]],
  [[0], [4, 2, 1, 0], [3, 0, 4, 1, 2]],
  [[3, 1, 0, 2], [1, 3, 2], [0, 2]],
  [[1], [3, 4, 2, 1, 0], []]],
 [[[3], [], [4, 0, 2, 1]],
  [[1, 4, 2], [0], [0, 3]],
  [[3, 1], [2, 3, 1], [4]],
  [[1], [4, 0, 1], []],
  [[2, 4, 0, 1, 3], [1, 4, 2], []],
  [[1, 0, 2], [0], [2, 0, 3, 1, 4]],
  [[0], [1, 0, 4, 2, 3], [2, 1, 0, 3]]])

# mutation

In [99]:
mutation_chance = 0.2

def mutate(chrome1):
    if random.uniform(0, 1) <= mutation_chance:
        randomDay = randrange(days)        
        [morningDoctors, eveningDoctors, nightDoctors] = sample(list(range(len(doctors_id)+1)), 3)
        
        chrome1[randomDay][0] = sample(list(range(len(doctors_id))), morningDoctors)
        chrome1[randomDay][1] = sample(list(range(len(doctors_id))), eveningDoctors)
        chrome1[randomDay][2] = sample(list(range(len(doctors_id))), nightDoctors)

        
chrome1 = population[0]
display(chrome1)
mutate(chrome1)

[[[3, 2, 1, 4], [2, 4, 3], [2]],
 [[], [4], [3, 2, 0, 3]],
 [[3], [0, 4, 1, 4, 2], []],
 [[4, 1, 1, 2, 4], [4, 3, 0], [2]],
 [[4], [3, 4, 3, 3], [2, 4, 2, 3, 4]],
 [[4, 4, 0, 1], [1, 4, 4], [1, 4]],
 [[3], [0, 3, 4, 1, 2], []]]

yaay


[[[3, 2, 1, 4], [2, 4, 3], [2]],
 [[], [4], [3, 2, 0, 3]],
 [[3], [0, 4, 1, 4, 2], []],
 [[4, 3, 2, 0], [0, 4], [3, 2, 4]],
 [[4], [3, 4, 3, 3], [2, 4, 2, 3, 4]],
 [[4, 4, 0, 1], [1, 4, 4], [1, 4]],
 [[3], [0, 3, 4, 1, 2], []]]

# Fitness

In [11]:
max_shift_num = 8

def calculate_fitness(chrome1):
    fitness = 0
    shift_numbers = dict.fromkeys(doctors_id, 0)
    for day in range(len(allShifts)):
        for shift in range(3):
            if allShifts[day][shift][0]< len(chrome1[day][shift]) < allShifts[day][shift][1]:
                fitness = fitness + 1
            for i in chrome1[day][shift]:
                shift_numbers[i] = shift_numbers[i] + 1

    for shift_num in shift_numbers.values():
        if shift_num <= max_shift_num:
                fitness = fitness + 1
                
    
            
    return fitness

chrome1 = population[0]
calculate_fitness(chrome1)

6

# Genetic